In [4]:
import os
import re
import csv
import pandas as pd
import numpy as np

# Arbeitsverzeichnis (NIST-Testdaten)
root_dir = r'data/nist'

data = []

for test_dir in os.listdir(root_dir):
    test_path = os.path.join(root_dir, test_dir)
    if not os.path.isdir(test_path):
        continue

    model_name = (re.search(r'Pi[^_]+', test_dir) or ["Unknown"])[0]
    
    for subdir, dirs, files in os.walk(test_path):
        for fname in ['PQCkemKAT.time', 'PQCencryptKAT.time']:
            if fname not in files:
                continue

            fp = os.path.join(subdir, fname)
            algorithm_name = os.path.basename(subdir)
            count = prepare = keypair = enc = dec = None

            with open(fp, encoding='utf-8') as f:
                for line in f:
                    line = line.strip()

                    # count
                    if line.startswith("count ="):
                        # vorherige Runde speichern
                        if count is not None:
                            data.append([model_name, algorithm_name, count,
                                         prepare, keypair, enc, dec])
                        count = int(line.split('=',1)[1])

                    # prepare
                    m = re.match(r'prepare \(μs\) = (\d+)', line)
                    if m:
                        prepare = int(m.group(1))

                    # keypair, enc, dec
                    m = re.match(r'crypto_(?:kem|encrypt)_(keypair|enc|dec) \(μs\) = (\d+)', line)
                    if m:
                        metric, val = m.group(1), int(m.group(2))
                        if metric == 'keypair':
                            keypair = val
                        elif metric == 'enc':
                            enc = val
                        elif metric == 'dec':
                            dec = val

                # letzte Runde speichern
                if count is not None:
                    data.append([model_name, algorithm_name, count,
                                 prepare, keypair, enc, dec])

# DataFrame bauen
df = pd.DataFrame(data, columns=[
    'Model', 'Algorithmus', 'count',
    'prepare (μs)', 'keypair (μs)', 'enc (μs)', 'dec (μs)'
])

# Aggregation wie gehabt…
time_columns = ['keypair (μs)', 'enc (μs)', 'dec (μs)']
grouped     = df.groupby(['Model','Algorithmus'])
df_agg      = grouped[time_columns].mean()
df_count    = grouped.size().rename("Runden")
df_combined = df_agg.join(df_count).reset_index()
df_combined['Total Time (μs)'] = df_combined[time_columns].sum(axis=1)
df_combined.sort_values(['Algorithmus','Model'], inplace=True)
df_combined = df_combined[['Model','Algorithmus','Runden'] + time_columns + ['Total Time (μs)']]

os.makedirs('output/csv', exist_ok=True)
df_combined.to_csv(
    'output/csv/nist_time_kem.csv',
    index=False,
    float_format="%.0f",
    quoting=csv.QUOTE_NONE
)

print(df_combined)


        Model    Algorithmus  Runden  keypair (μs)  enc (μs)  dec (μs)  \
0        Pi1b    BIG_QUAKE_1       5     6381259.4   19983.4   40427.2   
80   Pi1bPlus    BIG_QUAKE_1       5     6284844.4   19901.6   39248.2   
160      Pi3b    BIG_QUAKE_1       5     2152037.4    6137.4   10000.2   
287      Pi4b    BIG_QUAKE_1       5      659121.0    2925.8    4159.8   
414  PiBanana    BIG_QUAKE_1       5     2456860.0   10685.6   15058.6   
..        ...            ...     ...           ...       ...       ...   
286      Pi3b  newhope512cpa      10         396.5     631.1     188.2   
413      Pi4b  newhope512cpa      10          78.7     112.1      29.5   
485  PiBanana  newhope512cpa      10         590.3     761.2     119.0   
565    PiZero  newhope512cpa      10         906.3    1325.3     250.5   
692   PiZero2  newhope512cpa      10         264.0     391.6     117.7   

     Total Time (μs)  
0          6441670.0  
80         6343994.2  
160        2168175.0  
287         666206.